# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [7]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [2]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [6]:
def astar(problem: Problem):
    state = problem.initial_state
    queue = [(problem.heuristic(state),[], state, 0)]
    visited = []
    while queue:
        heuristic,path,state,cost = queue.pop(queue.index(min(queue, key=lambda x:x[0])))
#         print(heuristic,path,state,cost)
        if problem.is_goal(state):
#             print(cost)
#             print("Number of visited state: ",len(visited))
            return path, len(visited)
        if state in visited:
            continue
        visited.append(state)
        for action in problem.available_actions(state):
            new_state = problem.do_action(state,action)
            if new_state not in visited:
                move_cost = problem.action_cost(state,action)
                heuristic = problem.heuristic(new_state)
                new_cost = cost + move_cost +  heuristic
                new_path = path + [action]
                queue.append((new_cost,new_path,new_state,cost+move_cost))


In [4]:
path, num = astar(VacuumProblem())
print("Number of visited states: ", num)
path

Number of visited states:  5


['Suck', 'Right', 'Suck']

In [5]:
%timeit astar(VacuumProblem())

17 µs ± 736 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [284]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
path, num = astar(VacuumProblem1())
print("Number of visited states: ", num)
path

Number of visited states:  6


['Suck', 'Right', 'Suck']

In [185]:
%timeit astar(VacuumProblem1())

20.5 µs ± 395 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


And another in which heuristic grossly overestimates the cost.

In [283]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
path, num = astar(VacuumProblem2())
print("Number of visited states: ", num)
path

Number of visited states:  3


['Suck', 'Right', 'Suck']

In [265]:
%timeit astar(VacuumProblem2())

10.7 µs ± 296 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

By analyzing the results, the best heuristic is VacumProblem2, but it awards too many points for the action done and as a result favors the first 'Suck "move too much at the same time gives huge penalty to the actions (left / right), so it gets the best result.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [14]:
import numpy as np
import copy
from math import sqrt

class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        return [[7,2,4],
              [5,0,6],
                [8,3,1]]    
    
    def available_actions(self, state):
        return ['Up', 'Right', 'Down', 'Left']
        
    def do_action(self, state, action):
        new_state = copy.deepcopy(state)
        for x, y in enumerate(state):
            if 0 in y:
                row = x
                col = y.index(0)
        
        if action == 'Up':
            if row == 0:
                return state
            else:
                new_state[row][col] = new_state[row - 1][col]
                new_state[row - 1][col] = 0
                           
        elif action == 'Right':
            if col == 2:
                return state
            else:
                new_state[row][col] = state[row][col + 1]
                new_state[row][col + 1] = 0
                            
        elif action == 'Down':
            if row == 2:
                return state
            else:
                new_state[row][col] = state[row + 1][col]
                new_state[row + 1][col] = 0
                        
        elif action == 'Left':
            if col == 0:
                return state
            else:
                new_state[row][col] = state[row][col - 1]
                new_state[row][col - 1] = 0
                    
        return new_state
    
    def is_goal(self, state):
        goal = np.array([[0,1,2],
                        [3,4,5],
                        [6,7,8]])
        if (state == goal).all():
            return True
        else: 
            return False
        
        
    def action_cost(self, state, action):
        return 1
    
    def heurisitc_manhatan(self, state):
#         new_state = copy.deepcopy(state)
        b = np.array([[0,1,2],
                [3,4,5],
                [6,7,8]])
        a = np.array(state)
        dif = 0
        for elem in [x for x in range(0,8)]:
            i,j = np.where(a==elem)
            idx_state = (int(i),int(j))
            i,j = np.where(b==elem)
            idx_goal = (int(i),int(j))
        #     print(idx_state,idx_goal)
            dif += (abs(idx_state[0]-idx_goal[0])+abs(idx_state[1]-idx_goal[1]))
        return dif
    
    def heuristic_euclidean(self, state):
        b = np.array([[0,1,2],
                [3,4,5],
                [6,7,8]])
        a = np.array(state)
        dif = 0
        for elem in [x for x in range(0,9)]:
            i,j = np.where(a==elem)
            idx_state = (int(i),int(j))
            i,j = np.where(b==elem)
            idx_goal = (int(i),int(j))
        #     print(idx_state,idx_goal)
            dif += sqrt((idx_state[0]-idx_goal[0])**2 + (idx_state[1]-idx_goal[1])**2)
#         print(dif)
        return dif

    def heuristic(self, state):
        return self.heurisitc_manhatan(state)

In [15]:
path0, num0 = astar(PuzzleProblem())
print("Number of visited states: ", num0)
print(path0)

Number of visited states:  8996
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']


In [16]:
%timeit astar(PuzzleProblem())

20.1 s ± 283 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Prove that this heuristic is admissible.**

"heuristic_manhatan" is admissible because all any move can do is move one tile one step
closer to the goal. Manhattan distance is defined on a grid (only 4 moves are possible, no "across" moves LEFT, RIGHT, UP, DOWN) so it's similar to moves allowed in 8-puzzle problem and cost of each move is equal to 1.

In [17]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:
        return (self.heurisitc_manhatan(state)+self.heuristic_euclidean(state))/2

In [18]:
path1, num1 = astar(PuzzleProblem1())
print("Number of visited states: ", num1)
print(path1)

Number of visited states:  8235
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']


In [19]:
%timeit astar(PuzzleProblem1())

28.7 s ± 12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Prove that this heuristic is admissible.**

The average of Manhattan distance and euclidean distance is similar to the previous heuristic, also it's will never exceed the "real" distance.

In [20]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        return self.heuristic_euclidean(state)

In [21]:
path2, num2 = astar(PuzzleProblem2())
print("Number of visited states: ", num2)
print(path2)

Number of visited states:  8437
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']


In [22]:
%timeit astar(PuzzleProblem2())

45 s ± 504 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Prove that this heuristic is admissible.**

The Euclidean distance measure used in PuzzleProblem3 lower the actual travel length, because in our problem we can't move "across".
The score is calculate about Pythagoren theorem.

Run your heuristics on the given problem.

In [276]:
print("Is plan0==plan1?", path0 == path1)
print("Is plan0==plan2?", path0 == path2)
print("Is plan1==plan2?", path1 == path2)

Is plan0==plan1? True
Is plan0==plan2? True
Is plan1==plan2? True


**Which of the heuristics is the best for this task? Why is that?**

The best heuristic is the manhatan distance, it achieves the best time score. This method is the best for the our problem because it fits to the allowed moves in 8-puzzle an its count the "real" distance. If we could move "across" probly the best heuritic will be the average of sum Euclidean and Manhatan distance.

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.